In [46]:
# , parse_dates=['date_time']).rename(columns={'date_time': 'seconds'}
# vbox_VBoxXPCOMIPCD = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/logs/vbox_monitoring-VBoxXPCOMIPCD.csv', sep=';', decimal=',', dayfirst=False)
# vbox_monitoring_VBoxHeadless = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/logs/vbox_monitoring-VBoxHeadless.csv', parse_dates=['date_time']).rename(columns={'date_time': 'seconds'}, sep=';', decimal=',', dayfirst=False)

# response_times = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/logs/response_times.csv', parse_dates=['date_time']).rename(columns={'date_time': 'seconds'}, sep=';', decimal=',', dayfirst=False)
# reset_times = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/logs/reset_times.csv', parse_dates=['date_time']).rename(columns={'date_time': 'seconds'}, sep=';', decimal=',', dayfirst=False)

# machine_monitoring_zombies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/logs/machine_monitoring-zombies.csv', parse_dates=['date_time']).rename(columns={'date_time': 'seconds'}, sep=';', decimal=',', dayfirst=False)
# machine_monitoring_mem = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/logs/machine_monitoring-mem.csv', parse_dates=['date_time']).rename(columns={'date_time': 'seconds'}, sep=';', decimal=',', dayfirst=False)
# machine_monitoring_disk = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/logs/machine_monitoring-disk.csv', parse_dates=['date_time']).rename(columns={'date_time': 'seconds'}, sep=';', decimal=',', dayfirst=False)
# machineHost_server_status = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/logs/machineHost_server_status.csv', parse_dates=['date_time']).rename(columns={'date_time': 'seconds'}, sep=';', decimal=',', dayfirst=False)

try:
    import matplotlib.pyplot as plt
    import pandas as pd
    import scipy.stats as stats
    from sklearn.linear_model import LinearRegression

except ImportError as e:
    print(f'Erro de importação: {e}')
    exit(1)

machine_monitoring_cpu = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/logs/machine_monitoring-cpu.csv', sep=";", parse_dates=['date_time'])

machine_monitoring_cpu = machine_monitoring_cpu.set_index(machine_monitoring_cpu['date_time'])

machine_monitoring_cpu['tempo_passado'] = machine_monitoring_cpu['date_time'].dt.strftime('%H:%M:%S')
machine_monitoring_cpu['tempo_passado'] = abs(machine_monitoring_cpu.index - machine_monitoring_cpu.index[0]).total_seconds() / 3600

machine_monitoring_cpu
machine_monitoring_cpu.info()
print(lista_converter)

for coluna in machine_monitoring_cpu.columns:
  if coluna != 'date_time':
    machine_monitoring_cpu[coluna] = machine_monitoring_cpu[coluna].str.replace(",", ".").astype(float)

# Selecionar apenas as colunas numéricas para dividir
numeric_cols = machine_monitoring_cpu.select_dtypes(include='number').columns

# Divisão das colunas numéricas
machine_monitoring_cpu[numeric_cols] = machine_monitoring_cpu[numeric_cols].div(1)

incluirColunaY = True
tipoYlabel = '(Porcentagem)'
title = 'Monitoramento de Recursos da CPU'

for col in machine_monitoring_cpu.columns:
  if type(tipoYlabel) is str and incluirColunaY is True:
    parte_coluna_df = f"{col} {tipoYlabel}"

  else:
    parte_coluna_df = f"{tipoYlabel}"

  machine_monitoring_cpu[col] = machine_monitoring_cpu[col].fillna(0)

  x = machine_monitoring_cpu['tempo_passado'].values.reshape(-1, 1)
  y = machine_monitoring_cpu[col].values.reshape(-1, 1)

  model = LinearRegression()
  model.fit(x, y)

  Y_pred = model.predict(x)

  if type(tipoYlabel) is str:
    ylabel = parte_coluna_df

  elif type(tipoYlabel) is dict and col in tipoYlabel:
    ylabel = ylabel[col]

  else:
    ylabel = col


  if type(title) is str:
    title = title

  elif type(title) is dict and col in title:
    title = title[col]

  else:
    title = col




  # print(machine_monitoring_cpu.index)





  ax = machine_monitoring_cpu.plot(
      title=title,
      figsize=(10, 5),
      legend=0,
      y=col,
      xlabel='Time(H)',
      ylabel=ylabel,
      style='k'
  )


  # Adicionar a linha da regressão
  ax.plot(x, Y_pred, color='red')
  plt.show()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 377331 entries, 2024-08-03 20:01:47 to 2024-03-13 13:51:03
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   usr            377331 non-null  object        
 1   nice           377331 non-null  object        
 2   sys            377331 non-null  object        
 3   iowait         377331 non-null  object        
 4   soft           377331 non-null  object        
 5   date_time      377331 non-null  datetime64[ns]
 6   tempo_passado  377331 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 23.0+ MB
['usr', 'nice', 'sys', 'iowait', 'soft']


UFuncTypeError: ufunc 'matmul' did not contain a loop with signature matching types (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Float64DType'>) -> None